In [1]:
%load_ext autoreload
%autoreload 2
# add . to module name
import sys
sys.path.append('../src/')

In [2]:
from package.logger import Timed, rlog, setup
from package import storage
setup("INFO")

In [3]:
from mcr_py import run_mlc_with_bags, GraphCache
from copy import deepcopy
from typing import Any, Tuple

import pandas as pd
import geopandas as gpd

import mcr_py
from mcr_py import GraphCache
import pyrosm
from package import storage, strtime
from package.logger import Timed
from package.mcr.label import McRAPTORLabel, merge_intermediate_bags
from package.mcr.path import PathManager, PathType
from package.osm import osm, graph
from package.raptor.mcraptor_single import McRaptorSingle
from package.raptor.bag import Bag
from package.mcr.bag import (
    convert_mc_raptor_bags_to_intermediate_bags,
    convert_mlc_bags_to_intermediate_bags,
)

import matplotlib.pyplot as plt
from package.minute_city import minute_city
from package.mcr.data import MCRGeoData
from package.mcr.mcr import MCR

import os
import folium
from package.mcr.path import PathType

In [4]:
city_id = "Koeln"
stops_path = "../data/cleaned/stops.csv"
osm_path = osm.get_osm_path_from_city_id(city_id)

with Timed.info("Reading stops"):
	other_stops_df = storage.read_gdf(stops_path)

if not os.path.exists(osm_path) and city_id:
	rlog.info("Downloading OSM data")
	osm.download_city(city_id, osm_path)
else:
	rlog.info("Using existing OSM data")

osm_reader = osm.new_osm_reader(osm_path)

with Timed.info("Getting OSM graph"):
	nodes, edges = osm.get_graph_for_city_cropped_to_stops(osm_reader, other_stops_df)

[17:59:26] INFO     Reading stops                                                                   ]8;id=69500;file:///tmp/ipykernel_69231/1792129641.py\1792129641.py]8;;\:]8;id=479285;file:///tmp/ipykernel_69231/1792129641.py#5\5]8;;\

           INFO     Reading stops done (0.15 seconds)                                               ]8;id=11349;file:///tmp/ipykernel_69231/1792129641.py\1792129641.py]8;;\:]8;id=531034;file:///tmp/ipykernel_69231/1792129641.py#5\5]8;;\

           INFO     Using existing OSM data                                                        ]8;id=178951;file:///tmp/ipykernel_69231/1792129641.py\1792129641.py]8;;\:]8;id=570407;file:///tmp/ipykernel_69231/1792129641.py#12\12]8;;\

           INFO     Getting OSM graph                                                              ]8;id=247229;file:///tmp/ipykernel_69231/1792129641.py\1792129641.py]8;;\:]8;id=931861;file:///tmp/ipykernel_69231/1792129641.py#16\16]8;;\

           INFO     Loading OSM network from cache                                                 ]8;id=874288;file:///tmp/ipykernel_69231/1792129641.py\1792129641.py]8;;\:]8;id=553549;file:///tmp/ipykernel_69231/1792129641.py#16\16]8;;\

[17:59:27] INFO     Getting OSM graph done (0.94 seconds)                                          ]8;id=603596;file:///tmp/ipykernel_69231/1792129641.py\1792129641.py]8;;\:]8;id=880458;file:///tmp/ipykernel_69231/1792129641.py#16\16]8;;\

In [5]:
pois = minute_city.fetch_pois_for_area(nodes.unary_union.convex_hull, nodes)

In [6]:
stops_path = "../data/cleaned/stops.csv"
city_id = "Koeln"
osm_path = ""
structs = "../data/structs.pkl"

mcr_geo_data = MCRGeoData(
    stops_path, structs, city_id, bicycle_location_path="../data/bicycle_locations.csv"
)


[17:59:39] INFO     Reading stops                                                                        ]8;id=662904;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/mcr/data.py\data.py]8;;\:]8;id=396754;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/mcr/data.py#37\37]8;;\

           INFO     Reading stops done (0.04 seconds)                                                    ]8;id=292337;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/mcr/data.py\data.py]8;;\:]8;id=375791;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/mcr/data.py#37\37]8;;\

           INFO     Preparing graphs                                                                     ]8;id=975614;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/mcr/data.py\data.py]8;;\:]8;id=943858;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/mcr/data.py#40\40]8;;\

           INFO     Using existing OSM data                                                              ]8;id=415942;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/mcr/data.py\data.py]8;;\:]8;id=21803;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/mcr/data.py#40\40]8;;\

           INFO     Loading OSM network from cache                                                       ]8;id=285026;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/mcr/data.py\data.py]8;;\:]8;id=404433;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/mcr/data.py#40\40]8;;\

[17:59:56] INFO     Reading bicycle locations                                                            ]8;id=86008;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/mcr/data.py\data.py]8;;\:]8;id=482809;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/mcr/data.py#56\56]8;;\

           INFO     Reading bicycle locations done (0.04 seconds)                                        ]8;id=529176;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/mcr/data.py\data.py]8;;\:]8;id=722734;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/mcr/data.py#56\56]8;;\

[18:00:05] INFO     Preparing graphs done (25.90 seconds)                                                ]8;id=817462;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/mcr/data.py\data.py]8;;\:]8;id=650784;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/mcr/data.py#40\40]8;;\

           INFO     Creating graph cache                                                                ]8;id=239635;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/mcr/data.py\data.py]8;;\:]8;id=724972;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/mcr/data.py#133\133]8;;\

[18:00:09] INFO     Creating graph cache done (4.64 seconds)                                            ]8;id=856928;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/mcr/data.py\data.py]8;;\:]8;id=902935;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/mcr/data.py#133\133]8;;\

In [7]:
mcr_geo_data.add_pois_to_mm_graph(pois)

In [8]:
mcr = MCR(mcr_geo_data, disable_paths=False, enable_limit=False)
mcr.run(1638242852, "08:00:00", 2, output_path="../data/bags.pkl")

[18:00:11] INFO     Running Dijkstra step                                                              ]8;id=53340;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/logger.py\logger.py]8;;\:]8;id=442989;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/logger.py#37\37]8;;\

[18:00:13] INFO     Running Dijkstra step done (1.87 seconds)                                          ]8;id=333415;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/logger.py\logger.py]8;;\:]8;id=759255;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/logger.py#37\37]8;;\

           INFO     Extracting dijkstra step bags                                                      ]8;id=113013;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/logger.py\logger.py]8;;\:]8;id=562207;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/logger.py#37\37]8;;\

[18:00:18] INFO     Extracting dijkstra step bags done (4.77 seconds)                                  ]8;id=797195;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/logger.py\logger.py]8;;\:]8;id=967836;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/logger.py#37\37]8;;\

           INFO     Running iteration 1                                                                   ]8;id=797247;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/mcr/mcr.py\mcr.py]8;;\:]8;id=917929;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/mcr/mcr.py#97\97]8;;\

...

           INFO     Extracting walking step bags                                                       ]8;id=344437;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/logger.py\logger.py]8;;\:]8;id=280063;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/logger.py#37\37]8;;\

[18:02:00] INFO     Extracting walking step bags done (7.12 seconds)                                   ]8;id=757904;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/logger.py\logger.py]8;;\:]8;id=653572;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/logger.py#37\37]8;;\

           INFO     Saving bags                                                                        ]8;id=308500;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/logger.py\logger.py]8;;\:]8;id=778983;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/logger.py#37\37]8;;\

[18:02:11] INFO     Saving bags done (10.62 seconds)                                                   ]8;id=683805;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/logger.py\logger.py]8;;\:]8;id=517610;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/logger.py#37\37]8;;\

: 